## Data processing

Steps
1. Load cvs files
2. Merge files
3. correct data types
4. inputate NA

Notebook 03
5. Feature engeneering


Problems
date matching missing data for: only working for 2017
    kiwo dates 
    holiday 

Import libraries

In [1]:
import pandas as pd
import datetime

1. Load cvs files

In [2]:
# Load a CSV files
df_kiwo = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/kiwo.csv')
df_daten = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/umsatzdaten_gekuerzt.csv')
df_test = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/test.csv')
df_wetter = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Raw/wetter.csv')

In [5]:
#converts to date and sets index
df_kiwo['Datum'] = pd.to_datetime(df_kiwo['Datum'], format='%Y-%m-%d')

df_daten['Datum'] = pd.to_datetime(df_daten['Datum'], format='%Y-%m-%d')
df_test['Datum'] = pd.to_datetime(df_test['Datum'], format='%Y-%m-%d')

df_wetter['Datum'] = pd.to_datetime(df_wetter['Datum'], format='%Y-%m-%d')


Creates id ang warengruppe for day with Umsatz 0 and for test dates

2. Merge files

In [8]:
# merge dataframes on a common column
df_full = pd.merge(df_test, df_daten, on=['Datum', 'id', 'Warengruppe'], how='outer')

df_datenkiwo = pd.merge(df_full, df_kiwo, on='Datum', how='left')

df_merged = pd.merge(df_datenkiwo, df_wetter, on='Datum', how='left')

In [9]:
# Display the first 5 rows
print(df_merged.head())

        id      Datum  Warengruppe      Umsatz  KielerWoche  Bewoelkung  \
0  1307011 2013-07-01            1  148.828353          NaN         6.0   
1  1307012 2013-07-01            2  535.856285          NaN         6.0   
2  1307013 2013-07-01            3  201.198426          NaN         6.0   
3  1307014 2013-07-01            4   65.890169          NaN         6.0   
4  1307015 2013-07-01            5  317.475875          NaN         6.0   

   Temperatur  Windgeschwindigkeit  Wettercode  
0     17.8375                 15.0        20.0  
1     17.8375                 15.0        20.0  
2     17.8375                 15.0        20.0  
3     17.8375                 15.0        20.0  
4     17.8375                 15.0        20.0  


3. correct data types

In [10]:
#descriptive statistics
print(df_merged.describe())

#data types and non-null counts
print(df_merged.info())

#missing values in each column
print(df_merged.isnull().sum())

                 id                          Datum   Warengruppe       Umsatz  \
count  1.116400e+04                          11164  11164.000000  9334.000000   
mean   1.609066e+06  2016-07-13 18:31:28.355428352      3.088678   206.749044   
min    1.307011e+06            2013-07-01 00:00:00      1.000000     7.051201   
25%    1.501043e+06            2015-01-04 00:00:00      2.000000    96.897441   
50%    1.607148e+06            2016-07-14 12:00:00      3.000000   161.900831   
75%    1.801174e+06            2018-01-17 00:00:00      4.000000   280.644663   
max    1.907305e+06            2019-07-30 00:00:00      6.000000  1879.461831   
std    1.791961e+05                            NaN      1.488413   144.545189   

       KielerWoche    Bewoelkung    Temperatur  Windgeschwindigkeit  \
count        268.0  11029.000000  11083.000000         11083.000000   
mean           1.0      4.737964     12.021979            11.011008   
min            1.0      0.000000     -8.475000           

In [11]:
# set warengruppe, bewoelkung, windgeschwindigkeit, wetter code and kieler woche as integer
df_merged['Warengruppe'] = df_merged['Warengruppe'].astype('Int64')
df_merged['Bewoelkung'] = df_merged['Bewoelkung'].astype('Int64')
df_merged['Windgeschwindigkeit'] = df_merged['Windgeschwindigkeit'].astype('Int64')
df_merged['Wettercode'] = df_merged['Wettercode'].astype('Int64')
df_merged['KielerWoche'] = df_merged['KielerWoche'].astype('Int64')

Save to csv// import from csv

In [12]:
#save merged dataframe to a new CSV file
df_merged.to_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/merged_data.csv', index=False)

In [77]:
df_merged = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/merged_data.csv')

4. inputate NA

In [13]:
df_inputated = df_merged.copy()

In [17]:
#missing values in each column
print(df_inputated.isnull().sum())

id                         0
Datum                      0
Warengruppe                0
Umsatz                  1830
KielerWoche            10896
Bewoelkung               135
Temperatur                81
Windgeschwindigkeit       81
Wettercode                 0
Woche                      0
Monat                      0
dtype: int64


Categorical Wettercode per week mode

In [15]:
# derive week and month for df_inputated
df_inputated['Woche'] = pd.to_datetime(df_inputated['Datum']).dt.isocalendar().week
df_inputated['Monat'] = pd.to_datetime(df_inputated['Datum']).dt.month

# precompute weekly and monthly modes (ignore NaNs when computing mode)
weekly_mode_map = df_inputated.groupby('Woche')['Wettercode'].agg(
    lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else pd.NA
).to_dict()

monthly_mode_map = df_inputated.groupby('Monat')['Wettercode'].agg(
    lambda s: s.dropna().mode().iloc[0] if not s.dropna().mode().empty else pd.NA
).to_dict()

# fill missing Wettercode: use weekly mode, if not available fallback to monthly mode
def fill_wettercode_row(row):
    if pd.isna(row['Wettercode']):
        w_mode = weekly_mode_map.get(row['Woche'], pd.NA)
        if not pd.isna(w_mode):
            return w_mode
        return monthly_mode_map.get(row['Monat'], pd.NA)
    return row['Wettercode']

df_inputated['Wettercode'] = df_inputated.apply(fill_wettercode_row, axis=1)

In [16]:
# 1) Compare antes vs depois
print("ANTES:", df_merged['KielerWoche'].value_counts(dropna=False))
print("DEPOIS:", df_inputated['KielerWoche'].value_counts(dropna=False))

# 2) Quais linhas viraram 0?
flip = (df_merged['KielerWoche'] == 1) & (df_inputated['KielerWoche'] == 0)
print("Flips:", flip.sum())
print(df_merged.loc[flip, ['Datum']].assign(KW_before=1).head())

# 3) Veja se há múltiplas colunas de KielerWoche após os merges
[k for k in df_merged.columns if 'KielerWoche' in k]
[k for k in df_inputated.columns if 'KielerWoche' in k]


ANTES: KielerWoche
<NA>    10896
1         268
Name: count, dtype: Int64
DEPOIS: KielerWoche
<NA>    10896
1         268
Name: count, dtype: Int64
Flips: 0
Empty DataFrame
Columns: [Datum, KW_before]
Index: []


['KielerWoche']

set not kiwo to 0 

In [18]:
#fix kieler woche missing values by filling with 0
df_inputated['KielerWoche'].fillna(0, inplace=True)

#set 'KielerWoche' as integer (nullable int64 to allow current NaNs)
df_inputated['KielerWoche'] = df_inputated['KielerWoche'].astype('int64')

/tmp/ipykernel_22282/3859340180.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_inputated['KielerWoche'].fillna(0, inplace=True)


missing Bewoelkung, Temperatur and/or Windgeschwindigkeit
Inputated with mean of day after and before

In [19]:
#what days are missing Bewoelkung, Temperatur and/or     Windgeschwindigkeit
missing_weather = df_inputated[df_inputated[['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']].isnull().any(axis=1)]
print(missing_weather[[ 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']])

       Bewoelkung  Temperatur  Windgeschwindigkeit
6348         <NA>         NaN                 <NA>
6349         <NA>         NaN                 <NA>
6350         <NA>         NaN                 <NA>
6351         <NA>         NaN                 <NA>
6352         <NA>         NaN                 <NA>
...           ...         ...                  ...
10589        <NA>         NaN                 <NA>
10590        <NA>         NaN                 <NA>
10591        <NA>         NaN                 <NA>
10592        <NA>         NaN                 <NA>
10593        <NA>         NaN                 <NA>

[135 rows x 3 columns]


fill Bewoelkung, Temperatur, Windgeschwindigkeit to mean of day before and after

In [20]:
#fill missing weather data with the mean of day before and after
df_inputated['Bewoelkung'] = df_inputated['Bewoelkung'].interpolate()
df_inputated['Temperatur'] = df_inputated['Temperatur'].interpolate()
df_inputated['Windgeschwindigkeit'] = df_inputated['Windgeschwindigkeit'].interpolate()



In [21]:
# set warengruppe, bewoelkung, windgeschwindigkeit, wetter code and kieler woche as integer
df_inputated['Bewoelkung'] = df_inputated['Bewoelkung'].astype('int64')
df_inputated['Windgeschwindigkeit'] = df_inputated['Windgeschwindigkeit'].astype('int64')

In [22]:
# median values before and after imputation from 2016-12-11 to 2017-11-08
df_merged_ = df_merged[(df_merged['Datum'] >= '2016-12-11') & (df_merged['Datum'] <= '2017-11-08')]
df_inputated_ = df_inputated[(df_inputated['Datum'] >= '2016-12-11') & (df_inputated['Datum'] <= '2017-11-08')]

print("Mean Bewoelkung before imputation:", df_merged_['Bewoelkung'].mean())
print("Mean Bewoelkung after imputation:", df_inputated_['Bewoelkung'].mean())
print("Mean Temperatur before imputation:", df_merged_['Temperatur'].mean())
print("Mean Temperatur after imputation:", df_inputated['Temperatur'].mean())
print("Mean Windgeschwindigkeit before imputation:", df_merged['Windgeschwindigkeit'].mean())
print("Mean Windgeschwindigkeit after imputation:", df_inputated['Windgeschwindigkeit'].mean())

Mean Bewoelkung before imputation: 4.9167725540025415
Mean Bewoelkung after imputation: 4.863138686131387
Mean Temperatur before imputation: 12.191302867936118
Mean Temperatur after imputation: 12.044838182192763
Mean Windgeschwindigkeit before imputation: 11.011007849860146
Mean Windgeschwindigkeit after imputation: 11.007613758509494


Discrepancies of Bewoelkung, Temperatur and/or     Windgeschwindigkeit for the same day across WarenGruppe

In [23]:
# check if there are discrepancies for the same date for the columns 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit'
#for 'Warengruppe' 6 the temperature is different for the same date as the other 'Warengruppe'
discrepancies = df_inputated.groupby('Datum').agg({
    'Bewoelkung': pd.Series.nunique,
    'Temperatur': pd.Series.nunique,
    'Windgeschwindigkeit': pd.Series.nunique
}).reset_index() 
discrepancies = discrepancies[(discrepancies['Bewoelkung'] > 1) | (discrepancies['Temperatur'] > 1) | (discrepancies['Windgeschwindigkeit'] > 1)]
print("Discrepancies in weather data for the same date:")
print(discrepancies)

Discrepancies in weather data for the same date:
          Datum  Bewoelkung  Temperatur  Windgeschwindigkeit
1236 2016-12-11           1           6                    6
1526 2017-10-04           2           5                    1
1527 2017-10-05           2           5                    1
1553 2017-11-02           2           1                    1
1555 2017-11-04           2           1                    1
1557 2017-11-06           2           1                    1
1857 2018-09-08           1           5                    1
1858 2018-09-09           1           5                    1
1864 2018-09-15           1           5                    2
1865 2018-09-16           1           5                    2
1866 2018-09-17           1           5                    2
1867 2018-09-18           1           5                    2
1868 2018-09-19           1           5                    2
1869 2018-09-20           1           5                    2
1892 2018-10-13           1         

uses value of WarenGruppe 1 to all groups 

In [24]:
cols = ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']

for date in discrepancies['Datum']:
    weather_values = df_inputated.loc[df_inputated['Datum'] == date, ['Warengruppe'] + cols]
    reference_values = weather_values.loc[weather_values['Warengruppe'] == 1, cols].iloc[0]
    # >>> use .values para evitar alinhamento por rótulo <<<
    df_inputated.loc[df_inputated['Datum'] == date, cols] = reference_values.values


Save to csv// import from csv

In [25]:
#save inputated dataframe to a new CSV file

df_inputated.to_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/inputated_data.csv', index=False)

In [26]:
df_inputated = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/inputated_data.csv')